# 07 - Sistema de Recomendacoes
## Sprint 2 - Dia 8

**Objetivo:** Criar sistema de recomendacoes com 2 regras por cluster

**Hipotese H1:** Recomendacoes geram economia de 15-20%

**Restricao:** Exatamente 2 regras por cluster (8 regras no total)

## 1. Setup e Carregamento de Dados

In [17]:
import pandas as pd
import numpy as np
import json
import os

# Configuracoes
pd.set_option('display.float_format', lambda x: f'R$ {x:,.2f}' if abs(x) > 1 else f'{x:.2%}')
np.random.seed(42)

In [18]:
# Carregar dados
usuarios_clustered = pd.read_csv('../data/processed/usuarios_clustered.csv')
transacoes = pd.read_csv('../data/raw/transacoes.csv')

print(f"Usuarios: {len(usuarios_clustered)}")
print(f"Transacoes: {len(transacoes)}")
print(f"\nDistribuicao de Clusters:")
print(usuarios_clustered['cluster'].value_counts().sort_index())

Usuarios: 500
Transacoes: 194231

Distribuicao de Clusters:
cluster
0     59
1    196
2    167
3     78
Name: count, dtype: int64


## 2. Analise de Gastos por Categoria e Cluster

Vamos analisar os gastos medios por categoria para cada cluster, focando nas categorias nao essenciais.

In [3]:
# Filtrar apenas gastos (excluir Renda)
gastos = transacoes[transacoes['categoria'] != 'Renda'].copy()

# Merge com cluster
gastos = gastos.merge(usuarios_clustered[['user_id', 'cluster']], on='user_id')

print(f"Total de transacoes de gasto: {len(gastos)}")

Total de transacoes de gasto: 191231


In [4]:
# Definir categorias nao essenciais (maior potencial de economia)
CATEGORIAS_NAO_ESSENCIAIS = [
    'Alimentacao_Fora',
    'Vestuario',
    'Lazer',
    'Outros'
]

# Categorias essenciais (menor flexibilidade)
CATEGORIAS_ESSENCIAIS = [
    'Alimentacao_Casa',
    'Habitacao_Aluguel',
    'Habitacao_Contas',
    'Transporte',
    'Saude',
    'Educacao',
    'Telecomunicacoes',
    'Higiene_Limpeza'
]

print("Categorias nao essenciais (foco das recomendacoes):")
for cat in CATEGORIAS_NAO_ESSENCIAIS:
    print(f"  - {cat}")

Categorias nao essenciais (foco das recomendacoes):
  - Alimentacao_Fora
  - Vestuario
  - Lazer
  - Outros


In [5]:
# Calcular gasto mensal medio por usuario por categoria
gasto_mensal = gastos.groupby(['user_id', 'cluster', 'categoria', 'mes'])['valor'].sum().reset_index()
gasto_medio_usuario = gasto_mensal.groupby(['user_id', 'cluster', 'categoria'])['valor'].mean().reset_index()

# Agregar por cluster e categoria
gasto_por_cluster_categoria = gasto_medio_usuario.groupby(['cluster', 'categoria'])['valor'].agg(['mean', 'std', 'count']).reset_index()
gasto_por_cluster_categoria.columns = ['cluster', 'categoria', 'gasto_medio', 'gasto_std', 'num_usuarios']

print("Gasto medio mensal por cluster e categoria calculado!")

Gasto medio mensal por cluster e categoria calculado!


In [6]:
# Nomes dos clusters
NOMES_CLUSTERS = {
    0: 'Endividados Severos',
    1: 'Em Alerta',
    2: 'Endividados Moderados',
    3: 'Poupadores'
}

# Filtrar apenas categorias nao essenciais e criar pivot
gastos_nao_essenciais = gasto_por_cluster_categoria[
    gasto_por_cluster_categoria['categoria'].isin(CATEGORIAS_NAO_ESSENCIAIS)
].copy()

# Pivot para visualizacao
pivot_gastos = gastos_nao_essenciais.pivot(index='categoria', columns='cluster', values='gasto_medio')
pivot_gastos.columns = [NOMES_CLUSTERS[c] for c in pivot_gastos.columns]

print("\n=== GASTO MEDIO MENSAL POR CATEGORIA NAO ESSENCIAL (R$/mes) ===")
print(pivot_gastos.round(2).to_string())


=== GASTO MEDIO MENSAL POR CATEGORIA NAO ESSENCIAL (R$/mes) ===
                  Endividados Severos  Em Alerta  Endividados Moderados  Poupadores
categoria                                                                          
Alimentacao_Fora            R$ 619.91  R$ 306.75              R$ 432.53   R$ 472.94
Lazer                       R$ 232.53  R$ 113.96              R$ 167.86   R$ 170.53
Outros                      R$ 155.41   R$ 75.36              R$ 103.98   R$ 116.82
Vestuario                   R$ 294.00  R$ 154.14              R$ 207.73   R$ 212.20


In [7]:
# Identificar top categorias de gasto por cluster
print("\n=== TOP 2 CATEGORIAS NAO ESSENCIAIS POR CLUSTER ===")

for cluster_id in sorted(gastos_nao_essenciais['cluster'].unique()):
    cluster_data = gastos_nao_essenciais[gastos_nao_essenciais['cluster'] == cluster_id]
    top2 = cluster_data.nlargest(2, 'gasto_medio')
    
    print(f"\nCluster {cluster_id} - {NOMES_CLUSTERS[cluster_id]}:")
    for _, row in top2.iterrows():
        print(f"  {row['categoria']}: R$ {row['gasto_medio']:.2f}/mes")


=== TOP 2 CATEGORIAS NAO ESSENCIAIS POR CLUSTER ===

Cluster 0 - Endividados Severos:
  Alimentacao_Fora: R$ 619.91/mes
  Vestuario: R$ 294.00/mes

Cluster 1 - Em Alerta:
  Alimentacao_Fora: R$ 306.75/mes
  Vestuario: R$ 154.14/mes

Cluster 2 - Endividados Moderados:
  Alimentacao_Fora: R$ 432.53/mes
  Vestuario: R$ 207.73/mes

Cluster 3 - Poupadores:
  Alimentacao_Fora: R$ 472.94/mes
  Vestuario: R$ 212.20/mes


## 3. Definicao das Regras de Recomendacao

Cada cluster tera exatamente 2 regras baseadas nas categorias com maior potencial de economia.

In [8]:
# Definir regras de recomendacao
# Estrutura: categoria, percentual_reducao, mensagem

REGRAS_RECOMENDACAO = {
    0: {  # Endividados Severos - cortes drasticos
        'nome': 'Endividados Severos',
        'prioridade': 'CRITICA',
        'regras': [
            {
                'id': 'R0_1',
                'categoria': 'Alimentacao_Fora',
                'acao': 'reducao',
                'percentual': 0.70,
                'titulo': 'Cortar refeicoes fora de casa',
                'mensagem': 'Reduza drasticamente refeicoes fora de casa. Priorize marmitas e cozinhar em casa.',
                'dica': 'Planeje cardapio semanal e faca compras com lista'
            },
            {
                'id': 'R0_2',
                'categoria': 'Vestuario',
                'acao': 'eliminacao',
                'percentual': 0.90,
                'titulo': 'Suspender compras de vestuario',
                'mensagem': 'Suspenda compras de roupas nao essenciais por 3 meses.',
                'dica': 'Revise o guarda-roupa e conserte pecas antes de comprar novas'
            }
        ]
    },
    1: {  # Em Alerta - reducoes moderadas
        'nome': 'Em Alerta',
        'prioridade': 'MODERADA',
        'regras': [
            {
                'id': 'R1_1',
                'categoria': 'Alimentacao_Fora',
                'acao': 'reducao',
                'percentual': 0.40,
                'titulo': 'Reduzir refeicoes fora de casa',
                'mensagem': 'Limite refeicoes fora a 1-2x por semana.',
                'dica': 'Leve marmita para o trabalho pelo menos 3x por semana'
            },
            {
                'id': 'R1_2',
                'categoria': 'Lazer',
                'acao': 'limite',
                'percentual': 0.35,
                'titulo': 'Estabelecer teto para lazer',
                'mensagem': 'Defina um limite mensal para gastos com lazer.',
                'dica': 'Busque alternativas gratuitas: parques, eventos culturais, bibliotecas'
            }
        ]
    },
    2: {  # Endividados Moderados - cortes significativos
        'nome': 'Endividados Moderados',
        'prioridade': 'ALTA',
        'regras': [
            {
                'id': 'R2_1',
                'categoria': 'Alimentacao_Fora',
                'acao': 'reducao',
                'percentual': 0.50,
                'titulo': 'Reduzir significativamente refeicoes fora',
                'mensagem': 'Reduza refeicoes fora de casa pela metade.',
                'dica': 'Cozinhe em maior quantidade no fim de semana para a semana toda'
            },
            {
                'id': 'R2_2',
                'categoria': 'Vestuario',
                'acao': 'reducao',
                'percentual': 0.50,
                'titulo': 'Cortar gastos com vestuario',
                'mensagem': 'Reduza compras de roupas pela metade.',
                'dica': 'Compre apenas itens essenciais e aproveite promocoes'
            }
        ]
    },
    3: {  # Poupadores - otimizacoes
        'nome': 'Poupadores',
        'prioridade': 'BAIXA',
        'regras': [
            {
                'id': 'R3_1',
                'categoria': 'Transporte',
                'acao': 'otimizacao',
                'percentual': 0.15,
                'titulo': 'Otimizar gastos com transporte',
                'mensagem': 'Avalie alternativas de transporte mais economicas.',
                'dica': 'Considere caronas, transporte publico ou bicicleta para trajetos curtos'
            },
            {
                'id': 'R3_2',
                'categoria': 'Telecomunicacoes',
                'acao': 'revisao',
                'percentual': 0.20,
                'titulo': 'Revisar planos e assinaturas',
                'mensagem': 'Revise planos de celular, internet e streaming.',
                'dica': 'Cancele assinaturas nao utilizadas e negocie valores com operadoras'
            }
        ]
    }
}

print("Regras de recomendacao definidas!")
print(f"Total: {sum(len(v['regras']) for v in REGRAS_RECOMENDACAO.values())} regras (2 por cluster)")

Regras de recomendacao definidas!
Total: 8 regras (2 por cluster)


In [9]:
# Exibir regras formatadas
print("\n" + "="*70)
print("REGRAS DE RECOMENDACAO POR CLUSTER")
print("="*70)

for cluster_id, cluster_data in REGRAS_RECOMENDACAO.items():
    print(f"\n### Cluster {cluster_id}: {cluster_data['nome']} (Prioridade: {cluster_data['prioridade']})")
    print("-"*50)
    
    for i, regra in enumerate(cluster_data['regras'], 1):
        print(f"\n  Regra {i}: {regra['titulo']}")
        print(f"  Categoria: {regra['categoria']}")
        print(f"  Acao: {regra['acao']} ({regra['percentual']*100:.0f}%)")
        print(f"  Mensagem: {regra['mensagem']}")
        print(f"  Dica: {regra['dica']}")


REGRAS DE RECOMENDACAO POR CLUSTER

### Cluster 0: Endividados Severos (Prioridade: CRITICA)
--------------------------------------------------

  Regra 1: Cortar refeicoes fora de casa
  Categoria: Alimentacao_Fora
  Acao: reducao (70%)
  Mensagem: Reduza drasticamente refeicoes fora de casa. Priorize marmitas e cozinhar em casa.
  Dica: Planeje cardapio semanal e faca compras com lista

  Regra 2: Suspender compras de vestuario
  Categoria: Vestuario
  Acao: eliminacao (90%)
  Mensagem: Suspenda compras de roupas nao essenciais por 3 meses.
  Dica: Revise o guarda-roupa e conserte pecas antes de comprar novas

### Cluster 1: Em Alerta (Prioridade: MODERADA)
--------------------------------------------------

  Regra 1: Reduzir refeicoes fora de casa
  Categoria: Alimentacao_Fora
  Acao: reducao (40%)
  Mensagem: Limite refeicoes fora a 1-2x por semana.
  Dica: Leve marmita para o trabalho pelo menos 3x por semana

  Regra 2: Estabelecer teto para lazer
  Categoria: Lazer
  Acao: lim

## 4. Funcao para Gerar Recomendacoes

In [10]:
def calcular_gasto_medio_categoria(user_id, categoria, transacoes_df):
    """
    Calcula o gasto medio mensal de um usuario em uma categoria.
    """
    user_transacoes = transacoes_df[
        (transacoes_df['user_id'] == user_id) & 
        (transacoes_df['categoria'] == categoria)
    ]
    
    if len(user_transacoes) == 0:
        return 0.0
    
    # Agrupar por mes e calcular media
    gasto_mensal = user_transacoes.groupby('mes')['valor'].sum()
    return gasto_mensal.mean()


def gerar_recomendacoes(user_id, cluster, transacoes_df, regras=REGRAS_RECOMENDACAO):
    """
    Gera recomendacoes personalizadas para um usuario baseado em seu cluster.
    
    Parametros:
        user_id: ID do usuario
        cluster: Numero do cluster (0-3)
        transacoes_df: DataFrame com transacoes
        regras: Dicionario com regras de recomendacao
    
    Retorna:
        Lista com 2 recomendacoes personalizadas
    """
    if cluster not in regras:
        raise ValueError(f"Cluster {cluster} nao encontrado nas regras")
    
    cluster_regras = regras[cluster]
    recomendacoes = []
    
    for regra in cluster_regras['regras']:
        # Calcular gasto atual do usuario na categoria
        gasto_atual = calcular_gasto_medio_categoria(
            user_id, 
            regra['categoria'], 
            transacoes_df
        )
        
        # Calcular economia potencial
        economia_potencial = gasto_atual * regra['percentual']
        
        recomendacao = {
            'id': regra['id'],
            'titulo': regra['titulo'],
            'categoria': regra['categoria'],
            'acao': regra['acao'],
            'percentual_reducao': regra['percentual'],
            'gasto_atual': round(gasto_atual, 2),
            'economia_potencial': round(economia_potencial, 2),
            'mensagem': regra['mensagem'],
            'dica': regra['dica']
        }
        
        recomendacoes.append(recomendacao)
    
    return {
        'user_id': user_id,
        'cluster': cluster,
        'cluster_nome': cluster_regras['nome'],
        'prioridade': cluster_regras['prioridade'],
        'recomendacoes': recomendacoes,
        'economia_total': sum(r['economia_potencial'] for r in recomendacoes)
    }

print("Funcao gerar_recomendacoes() definida!")

Funcao gerar_recomendacoes() definida!


In [11]:
# Testar funcao com um usuario de cada cluster
print("\n" + "="*70)
print("TESTE: RECOMENDACOES PARA 1 USUARIO DE CADA CLUSTER")
print("="*70)

for cluster_id in range(4):
    # Pegar primeiro usuario do cluster
    user_id = usuarios_clustered[usuarios_clustered['cluster'] == cluster_id]['user_id'].iloc[0]
    
    # Gerar recomendacoes
    resultado = gerar_recomendacoes(user_id, cluster_id, gastos)
    
    print(f"\n### {resultado['cluster_nome']} (Prioridade: {resultado['prioridade']})")
    print(f"Usuario: {resultado['user_id']}")
    print("-"*50)
    
    for i, rec in enumerate(resultado['recomendacoes'], 1):
        print(f"\n  Recomendacao {i}: {rec['titulo']}")
        print(f"  Gasto atual: R$ {rec['gasto_atual']:.2f}/mes")
        print(f"  Economia potencial: R$ {rec['economia_potencial']:.2f}/mes ({rec['percentual_reducao']*100:.0f}%)")
    
    print(f"\n  >>> ECONOMIA TOTAL: R$ {resultado['economia_total']:.2f}/mes")


TESTE: RECOMENDACOES PARA 1 USUARIO DE CADA CLUSTER

### Endividados Severos (Prioridade: CRITICA)
Usuario: user_0002
--------------------------------------------------

  Recomendacao 1: Cortar refeicoes fora de casa
  Gasto atual: R$ 737.54/mes
  Economia potencial: R$ 516.28/mes (70%)

  Recomendacao 2: Suspender compras de vestuario
  Gasto atual: R$ 362.20/mes
  Economia potencial: R$ 325.98/mes (90%)

  >>> ECONOMIA TOTAL: R$ 842.26/mes

### Em Alerta (Prioridade: MODERADA)
Usuario: user_0004
--------------------------------------------------

  Recomendacao 1: Reduzir refeicoes fora de casa
  Gasto atual: R$ 342.10/mes
  Economia potencial: R$ 136.84/mes (40%)

  Recomendacao 2: Estabelecer teto para lazer
  Gasto atual: R$ 89.54/mes
  Economia potencial: R$ 31.34/mes (35%)

  >>> ECONOMIA TOTAL: R$ 168.18/mes

### Endividados Moderados (Prioridade: ALTA)
Usuario: user_0008
--------------------------------------------------

  Recomendacao 1: Reduzir significativamente refeicoe

## 5. Salvar Regras em JSON

In [12]:
# Preparar regras para salvar (sem funcoes, apenas dados)
regras_para_salvar = {
    'versao': '1.0',
    'data_criacao': '2026-01-26',
    'descricao': 'Regras de recomendacao de economia por cluster',
    'total_regras': 8,
    'clusters': REGRAS_RECOMENDACAO
}

# Salvar JSON
os.makedirs('../models', exist_ok=True)
caminho_json = '../models/recomendacoes_regras.json'

with open(caminho_json, 'w', encoding='utf-8') as f:
    json.dump(regras_para_salvar, f, ensure_ascii=False, indent=2)

print(f"Regras salvas em: {caminho_json}")

Regras salvas em: ../models/recomendacoes_regras.json


In [13]:
# Verificar arquivo salvo
with open(caminho_json, 'r', encoding='utf-8') as f:
    regras_carregadas = json.load(f)

print("Arquivo JSON carregado com sucesso!")
print(f"Versao: {regras_carregadas['versao']}")
print(f"Total de regras: {regras_carregadas['total_regras']}")
print(f"Clusters: {list(regras_carregadas['clusters'].keys())}")

Arquivo JSON carregado com sucesso!
Versao: 1.0
Total de regras: 8
Clusters: ['0', '1', '2', '3']


## 6. Resumo e Proximos Passos

In [14]:
# Calcular estatisticas das recomendacoes para todos os usuarios
print("\n" + "="*70)
print("RESUMO: ECONOMIA POTENCIAL POR CLUSTER")
print("="*70)

economias_por_cluster = {}

for cluster_id in range(4):
    usuarios_cluster = usuarios_clustered[usuarios_clustered['cluster'] == cluster_id]
    economias = []
    
    for user_id in usuarios_cluster['user_id']:
        resultado = gerar_recomendacoes(user_id, cluster_id, gastos)
        economias.append(resultado['economia_total'])
    
    economias_por_cluster[cluster_id] = {
        'nome': NOMES_CLUSTERS[cluster_id],
        'num_usuarios': len(economias),
        'economia_media': np.mean(economias),
        'economia_total': np.sum(economias),
        'economia_min': np.min(economias),
        'economia_max': np.max(economias)
    }

# Exibir resultados
for cluster_id, stats in economias_por_cluster.items():
    print(f"\nCluster {cluster_id}: {stats['nome']}")
    print(f"  Usuarios: {stats['num_usuarios']}")
    print(f"  Economia media: R$ {stats['economia_media']:.2f}/mes")
    print(f"  Economia total (cluster): R$ {stats['economia_total']:.2f}/mes")
    print(f"  Range: R$ {stats['economia_min']:.2f} - R$ {stats['economia_max']:.2f}")


RESUMO: ECONOMIA POTENCIAL POR CLUSTER

Cluster 0: Endividados Severos
  Usuarios: 59
  Economia media: R$ 698.53/mes
  Economia total (cluster): R$ 41213.48/mes
  Range: R$ 369.12 - R$ 1431.23

Cluster 1: Em Alerta
  Usuarios: 196
  Economia media: R$ 162.59/mes
  Economia total (cluster): R$ 31866.94/mes
  Range: R$ 75.90 - R$ 411.99

Cluster 2: Endividados Moderados
  Usuarios: 167
  Economia media: R$ 320.13/mes
  Economia total (cluster): R$ 53461.81/mes
  Range: R$ 170.20 - R$ 660.75

Cluster 3: Poupadores
  Usuarios: 78
  Economia media: R$ 120.90/mes
  Economia total (cluster): R$ 9430.02/mes
  Range: R$ 60.67 - R$ 252.17


In [15]:
# Calcular economia total projetada
economia_mensal_total = sum(e['economia_total'] for e in economias_por_cluster.values())
economia_anual_total = economia_mensal_total * 12

print("\n" + "="*70)
print("IMPACTO TOTAL PROJETADO")
print("="*70)
print(f"\nEconomia mensal total (500 usuarios): R$ {economia_mensal_total:,.2f}")
print(f"Economia anual total: R$ {economia_anual_total:,.2f}")
print(f"Economia media por usuario: R$ {economia_mensal_total/500:.2f}/mes")


IMPACTO TOTAL PROJETADO

Economia mensal total (500 usuarios): R$ 135,972.25
Economia anual total: R$ 1,631,667.00
Economia media por usuario: R$ 271.94/mes


In [16]:
# Checklist Dia 8
print("\n" + "="*70)
print("CHECKLIST DIA 8")
print("="*70)
print("\n[x] Carregar usuarios_clustered.csv e transacoes.csv")
print("[x] Analisar gastos por categoria e cluster")
print("[x] Definir 2 regras de recomendacao por cluster (8 total)")
print("[x] Criar funcao gerar_recomendacoes(user_id, cluster)")
print("[x] Salvar regras em models/recomendacoes_regras.json")
print("[x] Calcular economia potencial por cluster")
print("\n>>> DIA 8 CONCLUIDO!")
print("\nProximo passo (Dia 9): Calcular economia detalhada e validar H1")


CHECKLIST DIA 8

[x] Carregar usuarios_clustered.csv e transacoes.csv
[x] Analisar gastos por categoria e cluster
[x] Definir 2 regras de recomendacao por cluster (8 total)
[x] Criar funcao gerar_recomendacoes(user_id, cluster)
[x] Salvar regras em models/recomendacoes_regras.json
[x] Calcular economia potencial por cluster

>>> DIA 8 CONCLUIDO!

Proximo passo (Dia 9): Calcular economia detalhada e validar H1
